# Detector resolution
As you know, detectors are not perfect and do not provide an infinitely precise measurement. Instead they have resolutions that may be a constant value, or may depend on the measured value itself.
In this notebook we will explore the energy resolution of the LKr detector, and the time and momentum resolution of the Spectrometer.

The resolution is the difference that we may find between the measured value and a reference value (the "real" value). Unless working with simulation, we usually cannot know the exact real value. So we need to find either a reference value which was measured with a resolution that was much smaller than the resolution that we are trying to measure. This would give us an "exact" value for all practical purposes. Alternatively we can measure the resolution with respect to another measurement for which the resolution is already well known, or with respect to another measurement provided by the same detector (if this is practical). For these two last cases, the measured resolution will be the convolution of both measurements but the resolution of the tested detector can be obtained by deconvolution. 

In [ ]:
# As usual let's import all we need
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from na62 import prepare, hlf, extract, constants, stats
from lmfit import Model

In [ ]:
# Then we load the data
data, _ = prepare.import_root_files(["data/run12450.root"])

# LKr energy resolution
To measure the LKr energy resolution we can use one of the principles that we have seen in the PID notebook. We know that in almost 100% of the cases, electrons will leave all their energy inside the LKr. As a consequence if we can isolate a pure sample of electrons and knowing their energy (momenta), we have a reference for the energy that is supposed to be measured in the LKr. This can easily be provided by the Ke3 decay.

Let's first isolate a sample of Ke3, similarly to what we did in the PID notebook. However to improve the efficiency of our selection and increase both the statistics and the purity, we are not going to reject events that are in the $m_\text{miss}^2(\mu)$ peak (i.e. assuming muon mass for the track) but instead reject events where the track is associated to a MUV3 signal.

In [ ]:
# Define our momentum and missing mass squared peaks parameters, as in the PID notebook
p_center = 75000
p_sigma = 1100
mm2_mean = 0
mm2_sigma = 3000

# --- Define the selection cuts we will need ---
# Event topology
single_track_w_clusters_cond = hlf.make_exists_cut(["track1_exists", "cluster1_exists", "cluster2_exists"], ["track2_exists", "track3_exists"])
# Total momentum
ptot_condition = hlf.n(hlf.make_total_momentum_cut((p_center-5*p_sigma), (p_center+5*p_sigma)))
# Missing mass squared
in_ke3_peak_cond = hlf.make_missing_mass_sqr_cut((mm2_mean-3*mm2_sigma), mm2_mean+3*mm2_sigma, {"track1": constants.electron_mass, "cluster1": constants.photon_mass, "cluster2": constants.photon_mass})
# MUV3 signal
muv3_notmu_cond = hlf.make_muv3_cut(False, "track1")

In [ ]:
# Apply the cuts to the data
ke3_events = hlf.select(data, [single_track_w_clusters_cond, ptot_condition, in_ke3_peak_cond, muv3_notmu_cond])

In [ ]:
# TODO: add here selection quality plots for confirmation with MC. Use techniques from 04
# Find out which channel polutes

Let's have a look at the E/p distribution:

In [ ]:
ke3_events["track1_eop"].hist(bins=100)
plt.yscale("log")
plt.xlabel("E/p")
plt.title("E/p distribution for Ke3 candidates sample")

We can see that we do have some polution from another channel. However the electron peak is very clear and enough for our purpose. We want to fit the resolution for the tracks that are in the peak around 1. We can simply reject events with $E/p<0.8$ as those are clearly not electrons having been entirely absorbed inside the LKr

In [ ]:
eop_condition = hlf.make_eop_cut(0.8, None, "track1")
ke3_events = hlf.select(ke3_events, [eop_condition])

Let's now extract the expected track energy using the measured momentum, and compute the difference with respect to the measured energy.

In [ ]:
e = extract.track(ke3_events, 1)
e_energy = np.sqrt(e["momentum_mag"]**2 + constants.electron_mass**2)
delta_e = e["lkr_energy"] - e_energy

We can now plot and fit this difference. We will try to fit with a Gaussian model, and also with a double Gaussian model

In [ ]:
# Prepare a figure
ax = plt.figure().gca()

gauss_fit = stats.perform_fit(delta_e, bins=100, display_range=(-6000, 6000), ax=ax, model_wrapper=stats.gaussian_wrapper, fit_label="Single Gaussian")
gauss2_fit = stats.perform_fit(delta_e, bins=100, display_range=(-6000, 6000), ax=ax, model_wrapper=stats.gaussian2_wrapper, fit_label="Double Gaussian")
plt.xlabel("$\Delta E$ [MeV]")
plt.title(r"$\Delta E = E_\mathrm{LKr} - E_\mathrm{Straw}$ for electron sample")
print(f"Single Gaussian reduced Chi2: {gauss_fit.redchi:.2f}")
print(f"Double Gaussian reduced Chi2: {gauss2_fit.redchi:.2f}")

We can see in this case that the double Gaussian looks much better, both visually and comparing the reduced Chi2. We are therefore going to use the result of the double Gaussian model.

In [ ]:
gauss2_fit

We can conclude two things from this fit result:
 - The mean of both Gaussians are slightly but significantly shifted to negative value. This means that the measured energy is always a little less than what we can expect from the momentum measurement (but we cannot tell from this only which one, if any, is more correct).
 - The energy resolution, measured using the $\sigma$ of the dominating Gaussian is around $\sigma_{\Delta E} \sim 327 \pm 6$ MeV.

However we have to keep in mind that this resolution is the convolution of the LKr energy resolution and the Spectrometer momentum resolution.
$$\Delta E = E_\text{LKr} - E_\text{Straw}$$
$$\sigma_{\Delta E} = \sqrt{\sigma_{E_\text{LKr}}^2 + \sigma_{E_\text{Straw}}^2} \sim 327 \pm 6~\text{MeV}$$
We will however assume in the following that the Straw resolution is *small* with respect to the LKr resolution and can therefore be neglected at first order.

We also know that the resolution on the energy is not constant and depends on the energy itself. So we can select ranges of energy, repeat this procedure and extract the sigma for each range.

In [ ]:
# Bin the energies between 0 and 80 GeV in bins of 5 GeV
bin_size = 5000
cut = pd.cut(e_energy, np.arange(0,80000, bin_size), labels=False)

# Prepare arrays to store the fit results (bin center, sigma, fit error on sigma)
energy = []
values = []
err = []

# Loop over the bins in increasing order
for binID in cut.value_counts().sort_index().index:
    # Select only the tracks in the current bin
    delta_e_bin = delta_e.loc[cut==binID]

    # Remove bins containing too little stat (request at least 100 tracks in the bin)
    if(len(delta_e_bin)<=100):
        continue

    # Perform the fit, using the double Gaussian model
    fitr = stats.perform_fit(delta_e_bin, bins=100, display_range=(-6000, 6000), model_wrapper=stats.gaussian2_wrapper)

    # Fill the arrays with the results
    energy_val = binID*bin_size + bin_size/2.
    energy.append(energy_val)
    values.append(fitr.params["sigma"].value)
    err.append(fitr.params["sigma"].stderr)

# Transform the result arrays into numpy array for easier processing later
energy = np.array(energy)
values = np.array(values)
err = np.array(err)

In [ ]:
# Plot the relative resolution (sigma/E)
plt.errorbar(energy, values/energy, yerr=err/energy, capsize=2)
plt.title("Convoluted LKr $\oplus$ Straw energy resolution as a function of the energy")
plt.ylabel("$\sigma_E/E$")
plt.xlabel("E [MeV]")
plt.grid("both")

The curve above corresponds to the relative resolution $\frac{\sigma_E}{E}$. This can be parametrized as $\frac{\sigma_E}{E} = a \oplus \frac{b}{\sqrt{E}} \oplus \frac{c}{E} = a \oplus bE^{-1/2} \oplus cE^{-1}$ (with E in GeV to be able to compare with the official model). Let's fit the curve with this model.

In [ ]:
# Define the model 
def lkr_resolution_model(E, a, b, c):
    E = E/1000 # E in GeV for the model
    return np.sqrt(a**2 + (b/np.sqrt(E))**2 + (c/E)**2)

# Create the lmfit model
model = Model(lkr_resolution_model)
# Initial guess for the parameters (not unreasonable)
params = model.make_params(a = 0.01, b=0.01, c=0.01)

# Perform the fit
result = model.fit(values/energy, params, E=energy, weights=1/(err/energy))

# Plot the result
plt.errorbar(energy, values/energy, yerr=err/energy, capsize=2, label="Measured resolution")
plt.plot(energy, result.best_fit, '-', label='best fit')
plt.plot(energy, lkr_resolution_model(energy, 0.009, 0.048, 0.11), ':', label='Official resolution model')
plt.legend()
plt.title("Convoluted LKr $\oplus$ Straw energy resolution as a function of the energy")
plt.ylabel("$\sigma_E/E$")
plt.xlabel("E [MeV]")
plt.grid("both")

# Print the fit result
result

In [ ]:
print(f"a = {result.params['a'].value:.1%} +- {result.params['a'].stderr:.1%}")
print(f"b = {result.params['b'].value:.1%} +- {result.params['b'].stderr:.1%}")
print(f"c = {result.params['c'].value:.1%} +- {result.params['c'].stderr:.1%}")

The officially recognized resolution is 
$$\frac{\sigma_E}{E} = 0.9\% \oplus \frac{4.8\%}{\sqrt{E}} \oplus \frac{11\%}{E}$$
while our measurement provides:
$$\frac{\sigma_E}{E} = 0.6\% \oplus \frac{6.1\%}{\sqrt{E}} \oplus \frac{11.7\%}{E}$$

This result is not too bad, parameters 'a' and 'c' are very close to their official value and 'b' is only a couple of percents away (but within uncertainties).

# Spectrometer momentum resolution

## Using data
To measure the momentum resolution of the Spectrometer, we can use closed kinematics events involving tracks only (K3pi). In this way, we can select one of the tracks and reconstruct it from the rest of the event (the beam momentum and the two other tracks) to give us the expected momentum. This can then be compared with the measured momentum of the track. The resolution on this difference is a convolution of three times the Spectrometer momentum resolution (once for each track) and the beam momentum resolution.

In [ ]:
# Define our momentum and missing mass squared peaks parameters, as in the PID notebook
m_center = constants.kaon_charged_mass
m_sigma = 1

# --- Define the selection cuts we will need ---
# Event topology
three_tracks_cond = hlf.make_exists_cut(["track1_exists", "track2_exists", "track3_exists"], ["cluster1_exists", "cluster2_exists"])
# Invariant mass
ptot_condition = hlf.make_invariant_mass_cut((m_center-5*m_sigma), (m_center+5*m_sigma), mass_assignments={"track1": constants.pion_charged_mass, "track2": constants.pion_charged_mass, "track3": constants.pion_charged_mass})

In [ ]:
k3pi = hlf.select(data, [three_tracks_cond, ptot_condition])

In [ ]:
# Extract negative tracks and positive tracks
# Positive will be pos1 and pos2, arbitrarily, doesn't matter

# Extract tracks
t1 = hlf.set_mass(extract.track(k3pi, 1), constants.pion_charged_mass)
t2 = hlf.set_mass(extract.track(k3pi, 2), constants.pion_charged_mass)
t3 = hlf.set_mass(extract.track(k3pi, 3), constants.pion_charged_mass)

# Extract negatives from t1, t2, t3 and store the corresponding 2 positive tracks in their respective arrays.
# Then rotate and do it again (this is only to put that in a loop and avoid 3 repetitions of the code
# Need to do beam at the same time to take care of index misalignment
t_neg_arr = []
t_pos1_arr = []
t_pos2_arr = []
beam_arr = []
beam = extract.get_beam(k3pi)
t = [t1, t2, t3]
for i in range(3):
    neg_tracks = t[0]["charge"]==-1
    t_neg = t[0].loc[neg_tracks]
    t_pos1 = t[1].loc[neg_tracks]
    t_pos2 = t[2].loc[neg_tracks]
    t_neg_arr.append(t_neg)
    t_pos1_arr.append(t_pos1)
    t_pos2_arr.append(t_pos2)
    beam_arr.append(beam.loc[neg_tracks])
    t = [t[1], t[2], t[0]]

# Merge the dataframes in the arrays
t_neg = pd.concat(t_neg_arr)#.reset_index()
t_pos1 = pd.concat(t_pos1_arr)#.reset_index()
t_pos2 = pd.concat(t_pos2_arr)#.reset_index()
beam = pd.concat(beam_arr)

In [ ]:
# We will choose t_pos2 as the tested track because we want to test performances on a positive track. Which one doesn't matter
expected = hlf.three_vectors_sum([beam, hlf.three_vector_invert(t_neg), hlf.three_vector_invert(t_pos1)])
# And compute the delta between the expected and measured value
delta_p = expected["momentum_mag"] - t_pos2["momentum_mag"]

In [ ]:
gauss2_fit = stats.perform_fit(delta_p, bins=100, display_range=(-5000, 5000), fit_range=(-2000, 2000), plot=True, model_wrapper=stats.gaussian_wrapper, fit_label="Double Gaussian")
plt.xlabel("$\Delta p$ [MeV]")
plt.title(r"$\Delta p = p_\mathrm{K3pi} - p_\mathrm{Straw}$ for K3pi sample")
#plt.yscale("log")

In [ ]:
gauss2_fit

Fit result is ~1000 MeV but this is $\sigma^2 = \sigma^2_{t1} + \sigma^2_{t2} + \sigma^2_{t3} + \sigma^2_\mathrm{Beam} \approx 3\sigma^2_\mathrm{Straw} + \sigma^2_\mathrm{Beam}$
If we want the Straw resolution, we would have to take into account the Beam resolution:
$$\sigma_\mathrm{Straw} = \sqrt{\frac{\sigma^2 - \sigma^2_\mathrm{Beam}}{3}}$$

However the beam resolution is not known and can be quite large as it is only statistically measured over time. Let's make an exercise:  
Starting from the measured resolution from the fit above, let's apply the formula and estimate the 'true' Straw single track resolution as a function of the beam resolution.

In [ ]:
# Get the measured resolution
measured_sigma = gauss2_fit.params["sigma"].value

# Generate beam resolution from 0 to the measured resolution (cannot be higher, else the measured resolution would be higher)
beam_res = np.arange(0,measured_sigma)

# Compute the estimated Straw single track resolution using the formula and plot it
sigma_straw = np.sqrt((gauss2_fit.params["sigma"].value**2 - beam_res**2)/3)
plt.plot(beam_res, sigma_straw)
plt.grid("both")
plt.title("Estimated Straw single track resolution vs beam resolution")
plt.xlabel("$\sigma_\mathrm{Beam}$ [MeV]")
plt.ylabel("$\sigma_\mathrm{Straw}$ [MeV]")

We see that depending on the beam resolution we can actually estimate any Straw momentum resolution between ~550 MeV and 0 MeV.

Similarly to the LKr case, we can again compute the resolution in momentum bins. Doing this, we will record both the "raw" resolution (the sigma of $p_\text{exp}-p_\text{meas}$) but also the Straw "single-track resolution" using one of the most extreme case above where the raw resolution would be almost entirely due to the beam resolution ($\sigma_\text{Beam} = 870$ MeV).

In [ ]:
# Bin the momentum between 0 and 80 GeV in bins of 5 GeV
bin_size = 5000
cut = pd.cut(expected["momentum_mag"], np.arange(0,80000, bin_size), labels=False)

# Prepare arrays to store the fit results (bin center, sigma, sigma_single_track, fit error on sigma)
momentum = []
values = []
values_single = []
err = []

# Loop over the bins in increasing order
for binID in cut.value_counts().sort_index().index:
    # Select only the tracks in the current bin
    dp = delta_p.loc[cut==binID]

    # Remove bins containing too little stat (request at least 100 tracks in the bin)
    if(len(dp)<=100):
        continue

    # Perform the fit, using the single Gaussian model
    fitr = stats.perform_fit(dp, bins=100, display_range=(-3000, 3000))

    # Extract the single track sigma and fill the arrays with the results
    p = binID*bin_size + bin_size/2.
    momentum.append(p)
    sigma = fitr.params["sigma"].value
    sigma_beam = 870
    sigma_single = np.sqrt((sigma**2 - sigma_beam**2)/3)
    values.append(sigma)
    values_single.append(sigma_single)
    err.append(fitr.params["sigma"].stderr)

# Transform the result arrays into numpy array for easier processing later
momentum = np.array(momentum)
values = np.array(values)
values_single = np.array(values_single)
err = np.array(err)

In [ ]:
# Plot the relative resolution (sigma/E) for the fitted sigma (raw) and the single track sigma (computed)
plt.errorbar(momentum, values/momentum, yerr=err/momentum, capsize=2, label="Raw resolution ($P_\mathrm{exp} - P_\mathrm{meas}$)")
plt.errorbar(momentum, values_single/momentum, yerr=err/momentum, capsize=2, label="Single track resolution")
plt.legend()
plt.title("Extracted resolution of the Straw track in bins of momentum")
plt.xlabel("p [MeV]")
plt.ylabel("$\sigma_p$")
plt.grid("both")

The curve above corresponds to the relative resolution $\frac{\sigma_p}{p}$. This can be parametrized as $\frac{\sigma_p}{p} = a \oplus bp$ (with $p$ in GeV to be able to compare with the official model). Let's fit the curve with this model.

In [ ]:
# Define the model 
def straw_resolution_model(P, a, b):
    P = P/1000 # P in GeV for the model
    return np.sqrt(a**2 + (b*P)**2)

# Create the lmfit model
model = Model(straw_resolution_model)
# Initial guess for the parameters (not unreasonable)
params = model.make_params(a = 0.01, b=0.001)

# Perform the fit
result = model.fit(values_single/momentum, params, P=momentum, weights=1/(err/momentum))

# Plot the result
plt.errorbar(momentum, values_single/momentum, yerr=err/momentum, capsize=2, label="Measured resolution")
plt.plot(momentum, result.best_fit, '-', label='best fit')
plt.plot(momentum, straw_resolution_model(momentum, 0.003, 0.00005), ':', label='Official resolution model')
plt.legend()
plt.title("Extracted Straw momentum resolution as a function of the momentum")
plt.ylabel("$\sigma_p/p$")
plt.xlabel("p [MeV]")
plt.grid("both")

# Print the fit result
result

In [ ]:
print(f"a = {result.params['a'].value:.1%} +- {result.params['a'].stderr:.1%}")
print(f"b = {result.params['b'].value:.1%} +- {result.params['b'].stderr:.1%}")

The officially recognized resolution is 
$$\frac{\sigma_p}{p} = 0.3\% \oplus 0.005\%p$$
while our measurement provides:
$$\frac{\sigma_p}{p} = 0.5\% \oplus 0.0\%p$$

From the two plots above, it is very clear that the measured "raw" resolution is very far away from the official value. The functional shape of the curve is not even similar. This means that our measurement didn't work at all and is indeed dominated by the resolution on the reference value (i.e. probably the beam component). Taking into account a very high beam resolution, we reach something closer to the official value (parameter 'a' is only a fraction of percent away from the real value, but 'b' cannot be measured). Given the measurement shown on the plot above, it is however very doubtful that we can actually measure anything meaningful in this way.

We can use a different technique to extract the resolution from Monte-Carlo simulations

# From Simulation
Instead we are now going to use simulated data and use as reference the exact real momentum that was input in the simulation